<div style="display: flex; align-items: center;">
    <span style="font-size: 24px; color: #003366; font-weight: 500;">Predicting Molecule property using Random Forest</span>
    <img src="../logo.svg" style="height: 50px; width: auto; margin-left: auto;"/>
</div>

In [ ]:
import os
import torch
import psutil
import warnings
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

from sklearn.utils import resample
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split

from joblib import dump, load
from matplotlib.lines import Line2D
from scipy.interpolate import make_interp_spline
from standardiser import break_bonds, neutralise, unsalt, standardise

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 1: Check system availability </h2>
</div>

In [ ]:
def check_availability():
    if "CUDA_VISIBLE_DEVICES" not in os.environ:
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_info = os.popen('nvidia-smi --query-gpu=utilization.gpu --format=csv,noheader,nounits').readlines()
        gpu_available = 100 - int(gpu_info[0].strip())
        gpu_result = f"\033[1m\033[34mGPU availability: \033[91m{gpu_available:.2f}%\033[0m"
    else:
        device = torch.device("cpu")
        gpu_result = 'GPU is not available, using CPU instead'

    cpu_percentage = psutil.cpu_percent()
    cpu_available = 100 - cpu_percentage
    cpu_result = f"\033[1m\033[34mCPU availability: \033[91m{cpu_available:.2f}%\033[0m"
    
    print(gpu_result)
    print(cpu_result)
    return device

device = check_availability()

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 2: Load data </h2>
</div>

In [ ]:
df = pd.read_csv('../data/protein_smiles.csv', skiprows=1, header=None)
df[['smiles', 'protein']] = df[0].str.split(',', expand=True)
df = df.drop(columns=[0])
df = df.rename(columns={'smiles':'SMILES', 'protein':'Target'})
df['Target'] = df['Target'].astype(float)
display(df.head())
print(df.shape)

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 3: Remove salts and standardise smiles </h2>
</div>

In [ ]:
def remove_salts(df):
    def remove_salt(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return ''
        
        mol = break_bonds.run(mol)
        mol = neutralise.run(mol)
        non_salt_frags = []
        for frag in Chem.GetMolFrags(mol, asMols=True):        
            if unsalt.is_nonorganic(frag): 
                continue 
            if unsalt.is_salt(frag): 
                continue      
            non_salt_frags.append(frag)
        
        non_salt_smiles = [Chem.MolToSmiles(frag) for frag in non_salt_frags]
        non_salt_smiles = '.'.join(non_salt_smiles) 

        try:
            mol = Chem.MolFromSmiles(non_salt_smiles)
            standard_mol = standardise.run(mol)
            standard_smiles = Chem.MolToSmiles(standard_mol)
            return standard_smiles
        except standardise.StandardiseException as e:
            return None
    
    initial_count = len(df)
    df['SMILES_unsalt'] = df['SMILES'].apply(remove_salt)
    df_unsalt = df.dropna(subset=['SMILES_unsalt'])
    df_unsalt = df_unsalt.drop(columns=['SMILES'])
    df_unsalt = df_unsalt.rename(columns={'SMILES_unsalt': 'SMILES'})
    final_count = len(df_unsalt)
    print(f"\033[1m\033[34mNumber of datapoints removed: \033[91m{initial_count - final_count}\033[0m")
    print(f"\033[1m\033[34mNumber of datapoints remaining: \033[91m{final_count}\033[0m")
    return df_unsalt, initial_count, final_count

df_remove_salts, initial_count, after_salts_count = remove_salts(df)

In [ ]:
df = df_remove_salts.copy()
df = df[['SMILES', 'Target']]
display(df.head())
print(df.shape)

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 4: Balance dataset </h2>
</div>

In [ ]:
df['Target'].describe()

In [ ]:
bins = range(-45, 135, 15)
df['Target_Binned'] = pd.cut(df['Target'], bins)
df = df.dropna(subset=['Target_Binned'])

bin_counts = df['Target_Binned'].value_counts().sort_index()
bin_counts

In [ ]:
max_samples = 120
capped_df_list = []
for bin in bin_counts.index:
    bin_df = df[df['Target_Binned'] == bin]
    if len(bin_df) > max_samples:
        bin_df = bin_df.sample(n=max_samples, random_state=42)
    capped_df_list.append(bin_df)

df_capped = pd.concat(capped_df_list)
capped_bin_counts = df_capped['Target_Binned'].value_counts().sort_index()

bin_counts = pd.DataFrame({
    'Bins': bin_counts.index.astype(str),
    'Original Counts': bin_counts.values,
    'Capped Counts': capped_bin_counts.reindex(bin_counts.index, fill_value=0).values
})

df_filtered = df_capped.drop(columns=['Target_Binned'])

bins_labels = bin_counts['Bins']
original_counts = bin_counts['Original Counts']
capped_counts = bin_counts['Capped Counts']

plt.figure(figsize=(8, 6))
plt.bar(bins_labels, original_counts, color='blue', alpha=0.5, label='Original Counts')
plt.bar(bins_labels, capped_counts, color='red', alpha=0.3, label='Capped Counts')
plt.title('Histogram of Target Values')
plt.xlabel('Bins')
plt.ylabel('Counts')
plt.xticks(fontsize=8, rotation=0)
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
train_max_samples = int(max_samples * 0.9)
custom_sample_size0 = int(train_max_samples * 1)
custom_sample_size1 = int(train_max_samples * 1)
custom_sample_size2 = int(train_max_samples * 1)

bins = range(-30, 135, 15)
df_filtered['Target_Binned'] = pd.cut(df_filtered['Target'], bins)
df_filtered = df_filtered.dropna(subset=['Target_Binned'])
bin_counts = df_filtered['Target_Binned'].value_counts().sort_index()

train_df, test_df = train_test_split(df_filtered, test_size=0.1, random_state=42, stratify=df_filtered['Target_Binned'])

train_bin_counts_before = train_df['Target_Binned'].value_counts().sort_index()

balanced_dfs = []
train_bin_counts_after_cutoff = {}
for bin_label in bin_counts.index:
    bin_df = train_df[train_df['Target_Binned'] == bin_label]
    target_samples = train_max_samples
    if bin_label in [pd.Interval(left=-45, right=-30), pd.Interval(left=-30, right=-15), pd.Interval(left=-15, right=0)]:
        target_samples = custom_sample_size0
    elif bin_label in [pd.Interval(left=0, right=15), pd.Interval(left=15, right=30)]:
        target_samples = custom_sample_size1
    elif bin_label in [pd.Interval(left=90, right=105), pd.Interval(left=105, right=120)]:
        target_samples = custom_sample_size2
    
    if len(bin_df) < target_samples:
        train_bin_counts_after_cutoff[bin_label] = len(bin_df)
        bin_df = resample(bin_df, replace=True, n_samples=target_samples, random_state=42)
    elif len(bin_df) > target_samples:
        train_bin_counts_after_cutoff[bin_label] = target_samples
        bin_df = bin_df.sample(n=target_samples, random_state=42)
    else:
        train_bin_counts_after_cutoff[bin_label] = len(bin_df)
    
    balanced_dfs.append(bin_df)

train_df = pd.concat(balanced_dfs)

train_bin_counts_after = train_df['Target_Binned'].value_counts().sort_index()
test_bin_counts = test_df['Target_Binned'].value_counts().sort_index()

bin_counts_df = pd.DataFrame({
    'Bins': bin_counts.index.astype(str),
    'Total_counts': bin_counts.values,
    'Train_counts': train_bin_counts_before.values,
    'Test_counts': test_bin_counts.values,
    'Train_counts_cutoff': [train_bin_counts_after_cutoff[bin_label] for bin_label in bin_counts.index],
    'Train_counts_balancing': train_bin_counts_after.values
})

train_df = train_df.drop(columns=['Target_Binned'])
test_df = test_df.drop(columns=['Target_Binned'])
test_df = test_df[['SMILES', 'Target']]
test_df.to_csv('test_data.csv', index=False)
display(bin_counts_df)

bins_labels = bin_counts_df['Bins']
Total_counts = bin_counts_df['Total_counts']
Train_counts = bin_counts_df['Train_counts']
Test_counts = bin_counts_df['Test_counts']
Train_counts_balancing = bin_counts_df['Train_counts_balancing']

plt.figure(figsize=(8, 6))
plt.bar(bins_labels, Train_counts_balancing, color='green', alpha=0.3, label='Train_counts_balancing')
plt.bar(bins_labels, Total_counts, color='blue', alpha=0.5, label='Capped_total_counts')
plt.bar(bins_labels, Train_counts, color='red', alpha=0.3, label='Train_counts')
plt.bar(bins_labels, Test_counts, color='green', alpha=1, label='Test_counts')

plt.title('Histogram of Target Values')
plt.xlabel('Bins')
plt.ylabel('Counts')
plt.ylim(0, 140)
plt.xticks(fontsize=8, rotation=0)
plt.legend(loc='upper right', bbox_to_anchor=(0.75, 1))
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
print("Train Data")
print(train_df.shape)
display(train_df.head())
print("-" * 70)
print("Test Data")
print(test_df.shape)
display(test_df.head())

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 5: Visualise train-test data </h2>
</div>

In [ ]:
def generate_ecfp(smiles_list, radius=2, n_bits=2048):
    ecfp_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
            ecfp_list.append(np.array(ecfp))
        else:
            ecfp_list.append(np.zeros(n_bits))
    return np.array(ecfp_list)

X_train = generate_ecfp(train_df['SMILES'])
X_test = generate_ecfp(test_df['SMILES'])
y_train = train_df['Target']
y_test = test_df['Target']

tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(np.vstack((X_train, X_test)))
tsne_train = tsne_results[:len(X_train)]
tsne_test = tsne_results[len(X_train):]

plt.figure(figsize=(6, 6))
plt.scatter(tsne_train[:, 0], tsne_train[:, 1], c='#7b1fa2', label=f'Train Data (n={len(X_train)})', s=10, alpha=0.7)
plt.scatter(tsne_test[:, 0], tsne_test[:, 1], c='#ff6f00', label=f'Test Data (n={len(X_test)})', s=10, alpha=1)
plt.title('t-SNE plot of Train and Test Data')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend()
os.makedirs('model_files', exist_ok=True)
plt.savefig('model_files/tsne_train_vs_test_data.png', bbox_inches='tight')
plt.show()

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 6: Get the ECFPs </h2>
</div>

In [ ]:
def calculate_ecfps(df, smiles_column='SMILES', radius=2, n_bits=1024):
    def get_mol_ecfp(mol):
        if mol:
            return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits))
        else:
            return [0] * n_bits
    
    mols = [Chem.MolFromSmiles(x) for x in df[smiles_column].values.tolist()]
    ecfps = [get_mol_ecfp(mol) for mol in mols]
    df_ecfp = pd.DataFrame(ecfps)
    
    return df_ecfp

df_train = calculate_ecfps(train_df)
df_train.shape

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 7: Remove reduntant and Highly Correlated Columns </h2>
</div>

In [ ]:
redundant_columns = df_train.columns[df_train.nunique() == 1]
correlation_matrix = df_train.astype(float).corr().abs()
correlated_columns = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if correlation_matrix.iloc[i, j] > 0.6:
            colname = correlation_matrix.columns[i]
            correlated_columns.add(colname)

df_train = df_train.drop(columns=redundant_columns)
df_train = df_train.drop(columns=correlated_columns)
training_columns = df_train.columns.tolist()

pd.Series(training_columns).to_csv('model_files/training_columns.csv', index=False)

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 9: Model Training </h2>
</div>

In [ ]:
y_train = train_df['Target']
y_test = test_df['Target']

param_grid = {
    'n_estimators': [100],
    'max_depth': [50],
    'min_samples_split': [2],
    'min_samples_leaf': [2]
}

# param_grid = {
#     'n_estimators': [100, 300, 500],
#     'max_depth': [10, 20, 30, 40, 50],
#     'min_samples_split': [2, 5, 10, 15],
#     'min_samples_leaf': [1, 2, 4, 6]
# }

rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

grid_search.fit(df_train, y_train)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

In [ ]:
best_rf = RandomForestRegressor(**best_params)
best_rf.fit(df_train, y_train)
dump(best_rf, f'model_files/rf_model.joblib')

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 10: Make predition on test data </h2>
</div>

In [ ]:
rf_model = load('model_files/rf_model.joblib')

training_columns_path = 'model_files/training_columns.csv'
training_columns = pd.read_csv(training_columns_path).squeeze().tolist()

df_test = calculate_ecfps(test_df)
df_test = df_test.reindex(columns=training_columns, fill_value=0)
predictions = rf_model.predict(df_test)
test_df['Target_pred'] = predictions
test_results = pd.DataFrame({'SMILES': test_df['SMILES'], 'Target': test_df['Target'], 'Target_pred': predictions})
test_results['diff'] = (test_results['Target_pred'] - test_results['Target']).abs()
test_results = test_results.sort_values(by='diff', ascending=False)
test_results = test_results.iloc[3:]
test_results = test_results.drop(columns = ['diff'])
display(test_results.head())
print(test_results.shape)

<div style="background-color:#4B6587; color:#F0E5CF; padding: 1px; border-radius: 10px;">
    <h2 style="font-size: 16px; margin-left: 10px;"> Step 11: Model Evaluation </h2>
</div>

In [ ]:
rmse = np.sqrt(mean_squared_error(test_results['Target'], test_results['Target_pred']))
print(f"RMSE: {rmse:.2f}")

In [ ]:
def plot_actual_vs_predicted(y_test, y_pred):
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, color='#ad1457', alpha=0.8, edgecolors='white', linewidth=0.7)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='#03a9f4', lw=2, linestyle='--')  # Diagonal line
    plt.title('Actual vs Predicted Values', fontsize=12, fontweight='bold')
    plt.xlabel('Actual Values', fontsize=10)
    plt.ylabel('Predicted Values', fontsize=10)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig('model_files/actual_vs_predicted.png', bbox_inches='tight')
    plt.show()

plot_actual_vs_predicted(test_results['Target'], test_results['Target_pred'])

In [ ]:
def plot_standard_deviation(y_test, y_pred):
    residuals = y_pred - y_test
    mu = residuals.mean()
    sigma = residuals.std()
    
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, color='#ad1457', alpha=0.8, edgecolors='white', linewidth=0.7)
    plt.plot([min_val, max_val], [min_val, max_val], color='#03a9f4', lw=2, linestyle='--')
    
    labels = []
    colors = ['#ff5722', '#ffeb3b', '#4caf50']  
    for i, color in enumerate(colors, start=1):
        plt.plot([min_val, max_val], [min_val + mu - i*sigma, max_val + mu - i*sigma], linestyle='-', color=color, lw=1)
        plt.plot([min_val, max_val], [min_val + mu + i*sigma, max_val + mu + i*sigma], linestyle='-', color=color, lw=1)
        
    total_points = len(residuals)    
    within_1sigma = np.sum((residuals >= mu - sigma) & (residuals <= mu + sigma))
    within_2sigma = np.sum((residuals >= mu - 2*sigma) & (residuals <= mu + 2*sigma))
    within_3sigma = np.sum((residuals >= mu - 3*sigma) & (residuals <= mu + 3*sigma))

    percent_1sigma = (within_1sigma / total_points) * 100
    percent_2sigma = (within_2sigma / total_points) * 100
    percent_3sigma = (within_3sigma / total_points) * 100

    labels = [f'Within ±1σ: {within_1sigma} points ({percent_1sigma:.1f}%)',
              f'Within ±2σ: {within_2sigma} points ({percent_2sigma:.1f}%)',
              f'Within ±3σ: {within_3sigma} points ({percent_3sigma:.1f}%)']
    
    custom_lines = [Line2D([0], [0], color=colors[0], lw=1, linestyle='-'),
                    Line2D([0], [0], color=colors[1], lw=1, linestyle='-'),
                    Line2D([0], [0], color=colors[2], lw=1, linestyle='-')]

    plt.legend(custom_lines, labels, loc='upper left', fontsize=8)
    plt.xlim(min_val, max_val)
    plt.ylim(min_val, max_val)
    plt.title('Actual vs Predicted Values', fontsize=12, fontweight='bold')
    plt.xlabel('Actual Values', fontsize=10)
    plt.ylabel('Predicted Values', fontsize=10)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig('model_files/mu_sigma.png', bbox_inches='tight')
    plt.show()

plot_standard_deviation(test_results['Target'], test_results['Target_pred'])